<a href="https://colab.research.google.com/github/NeuroCodeInnovations/Estimacion_Peso_Cuyes/blob/main/ResNet50/ResNet50_Estimacion_Cuyes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimación del peso del cuy usando ResNet


# Instalaciones previas

Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importar librerias

In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import matplotlib.pyplot as plt

# Cargar e inspeccionar los datos

In [ ]:
weights_path = '/content/drive/MyDrive/IMAG_CUYES/weights_train.txt'
weights_df = pd.read_csv(weights_path, header=None, names=['filename', 'weight'])
weights_df['weight'] = weights_df['weight'].astype(float)

In [ ]:
print(weights_df.head())

              filename     weight
0  B_CUY_01_001_01.jpg  308.44256
1  B_CUY_01_001_02.jpg  308.44256
2  B_CUY_01_001_03.jpg  308.44256
3  B_CUY_01_001_04.jpg  308.44256
4  B_CUY_01_001_05.jpg  308.44256


# Transformar los datos
Función para crear la clase DataSet de Pytorch para el entrenamiento

In [ ]:
class GuineaPigDataset(Dataset):
    def __init__(self, images_folder, weights_df, transform=None):
        self.images_folder = images_folder
        self.weights_df = weights_df
        self.transform = transform

    def __len__(self):
        return len(self.weights_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_folder, self.weights_df.iloc[idx, 0])

        # Verifica si el archivo existe
        if not os.path.exists(img_name):
            print(f"Archivo no encontrado: {img_name}")

        image = Image.open(img_name).convert("RGB")
        weight = self.weights_df.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(weight, dtype=torch.float32)

In [ ]:
# Transformación y carga de datos
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # ResNet espera imágenes de 256x256
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # Normalización para ResNet (Saber por qué)
])

# Dataset para el entrenamiento
train_data = GuineaPigDataset('/content/drive/MyDrive/IMAG_CUYES/train', weights_df, transform=transform)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

# Modificar ResNet para regresión

Usaremos ResNet-50 y reemplazaremos la última capa para que tenga una salida única

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Cargar el modelo preentrenado ResNet-50
model = models.resnet50(pretrained=True)

# Modificar la última capa para regresión
model.fc = nn.Linear(model.fc.in_features, 1)  # 1 salida para el peso

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 198MB/s]


# Configurar el entrenamiento

Definiremos el optimizador, la función de pérdida y el ciclo de entrenamiento. Para regresión, usamos el error cuadrático medio (MSE) como función de pérdida.

In [ ]:
import torch.optim as optim

# Configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

if torch.cuda.is_available():
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
else:
    print("No hay GPU disponible, se utilizará la CPU.")

# Definir la función de pérdida y el optimizador
criterion = nn.MSELoss()  #(Error Cuadrático Medio) mide la diferencia promedio al cuadrado entre las predicciones del modelo y los valores reales.
optimizer = optim.Adam(model.parameters(), lr=0.001)

GPU disponible: Tesla T4


# Entrenar el modelo



In [ ]:
num_epochs = 3  # Ajusta según el rendimiento y los recursos disponibles

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, weights in train_loader:
        images, weights = images.to(device), weights.to(device).unsqueeze(1)  # Esto convierte weights de [batch] a [batch, 1]

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, weights)

        # Backward y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    # Guardar el modelo en cada época
    torch.save(model.state_dict(), f'/content/drive/MyDrive/EPOCHS/02/weights_epoch_{epoch+1}.pth')

Epoch [1/3], Loss: 331307.6879
Epoch [2/3], Loss: 95369.1348
Epoch [3/3], Loss: 87576.6271


# Evaluar el modelo

Cargar el modelo entrenado

In [ ]:
# Suponiendo que tu modelo se guarda como 'model.pth'
model = models.resnet50(pretrained=False)  # Ajusta esto si usaste una arquitectura diferente
num_classes = 1  # Cambia esto según el número de clases (en tu caso, pesos)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Carga los pesos del modelo
model.load_state_dict(torch.load('/content/drive/MyDrive/EPOCHS/02/weights_epoch_3.pth'))
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-6-ef8c4c156b5c>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Preparar las transformaciones de la imagen

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ajusta el tamaño según el modelo
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Valores típicos para modelos preentrenados
])

Función para evaluar y predecir

In [ ]:
def evaluate_model(model, image_folder):
    predictions = []

    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        img = img.unsqueeze(0)  # Añadir dimensión de batch

        with torch.no_grad():
            output = model(img)
            prediction = output.item()  # Obtener el valor escalar
            predictions.append((img_name, prediction))

    return predictions

# Ejecutar la evaluación

Llama a la función y pasa la carpeta de imágenes de prueba.

In [ ]:
image_folder = '/content/drive/MyDrive/IMAG_CUYES/test'
predictions = evaluate_model(model, image_folder)

# Convertir a un DataFrame para facilitar la visualización
predictions_df = pd.DataFrame(predictions, columns=['Imagen', 'Predicción (gramos)'])
print(predictions_df)


                Imagen  Predicción (gramos)
0    CUY_01_024_08.jpg           563.004883
1    CUY_01_024_01.jpg           620.318359
2    CUY_00_217_01.jpg           922.946777
3    CUY_01_024_04.jpg           535.084106
4    CUY_01_024_05.jpg           672.877136
..                 ...                  ...
450  CUY_01_035_01.jpg           599.757935
451  CUY_01_026_13.jpg           538.565308
452  CUY_01_026_11.jpg           528.808472
453  CUY_01_026_03.jpg           663.990479
454  CUY_01_035_05.jpg           708.845215

[455 rows x 2 columns]


# Evaluar la precisión

Comparar las predicciones con los pesos reales para evaluar el rendimiento del modelo.

In [ ]:
# Cargar los pesos reales
weights_df = pd.read_csv('/content/drive/MyDrive/IMAG_CUYES/weights_test.txt', sep=',', header=None, names=['Imagen', 'Peso'])
merged_df = pd.merge(predictions_df, weights_df, on='Imagen', how='inner')
merged_df['Error'] = merged_df['Predicción (gramos)'] - merged_df['Peso']

print(merged_df.head())  # Muestra las primeras filas del DataFrame
#print(merged_df.info())  # Información sobre el DataFrame (incluyendo conteo de nulos)

              Imagen  Predicción (gramos)       Peso       Error
0  CUY_01_024_08.jpg           563.004883   771.1064 -208.101517
1  CUY_01_024_01.jpg           620.318359   771.1064 -150.788041
2  CUY_00_217_01.jpg           922.946777  1014.0000  -91.053223
3  CUY_01_024_04.jpg           535.084106   771.1064 -236.022294
4  CUY_01_024_05.jpg           672.877136   771.1064  -98.229264


Guardar los resultados de la prueba

In [ ]:
# Guardar el DataFrame en un archivo .txt
ruta = '/content/drive/MyDrive/RESULTADOS/02/resultados_errores.txt'
merged_df.to_csv(ruta, sep='\t', index=False)

print("Archivo 'resultados_errores.txt' generado exitosamente.")

Archivo 'resultados_errores.txt' generado exitosamente.


Metricas

In [ ]:
# Calcular métricas como el error cuadrático medio (RMSE)
rmse = (merged_df['Error']**2).mean()**0.5
print(f'RMSE: {rmse:.4f}')


# Calculamos el MAE (Mean Absolute Error)
mae = (merged_df['Predicción (gramos)'] - merged_df['Peso']).abs().mean()
print(f'MAE: {mae:.4f}')

RMSE: 285.6666
MAE: 226.2983
